In [13]:
import sys, os
sys.path.append(os.path.expanduser('/home/dnelson/project/msprime/'))
import msprime
import math

In [19]:
c_m = 2 # Would be 1 in monogamous model
p = 1 # Cousin-ship
N = 100000
K = 10000

prob_pair_are_cousins = c_m * (2 ** (p + 1) - 1) * (2 ** p) / N
num_cousin_pairs = prob_pair_are_cousins * (K ** 2) / 2

print(prob_pair_are_cousins)
print(num_cousin_pairs)

0.00012
6000.0


In [14]:
def out_of_africa(nhaps):
    """
    Specify the demographic model used in these simulations (Gravel et al, 2011 PNAS)
    """
    # First we set out the maximum likelihood values of the various parameters
    # given in Gravel et al, 2011 Table 2.
    N_A = 7300
    N_B = 1861
    N_AF = 14474
    N_EU0 = 1032
    N_AS0 = 554
    # Times are provided in years, so we convert into generations.
    generation_time = 25
    T_AF = 148e3 / generation_time
    T_B = 51e3 / generation_time
    T_EU_AS = 23e3 / generation_time
    # We need to work out the starting (diploid) population sizes based on
    # the growth rates provided for these two populations
    r_EU = 0.0038
    r_AS = 0.0048
    N_EU = N_EU0 / math.exp(-r_EU * T_EU_AS)
    N_AS = N_AS0 / math.exp(-r_AS * T_EU_AS)
    # Migration rates during the various epochs.
    m_AF_B = 15e-5
    m_AF_EU = 2.5e-5
    m_AF_AS = 0.78e-5
    m_EU_AS = 3.11e-5

    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU and 2=CHB
    # initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=nhaps[0], initial_size=N_AF),
        msprime.PopulationConfiguration(
            sample_size=nhaps[1], initial_size=N_EU, growth_rate=r_EU),
        msprime.PopulationConfiguration(
            sample_size=nhaps[2], initial_size=N_AS, growth_rate=r_AS)
    ]
    migration_matrix = [
        [      0, m_AF_EU, m_AF_AS],
        [m_AF_EU,       0, m_EU_AS],
        [m_AF_AS, m_EU_AS,       0],
    ]
    demographic_events = [
        # CEU and CHB merge into B with rate changes at T_EU_AS
        msprime.MassMigration(
            time=T_EU_AS, source=2, destination=1, proportion=1.0),
        msprime.MigrationRateChange(time=T_EU_AS, rate=0),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
        # Population B merges into YRI at T_B
        msprime.MassMigration(
            time=T_B, source=1, destination=0, proportion=1.0),
        # Size changes to N_A at T_AF
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    # Use the demography debugger to print out the demographic history
    # that we have just described.
    dp = msprime.DemographyDebugger(
        Ne=N_A,
        population_configurations=population_configurations,
        migration_matrix=migration_matrix,
        demographic_events=demographic_events)
    dp.print_history()

    return(population_configurations, migration_matrix, demographic_events)



In [15]:
out_of_africa([100, 100, 100])

Creating ll_sim
Done
Model =  hudson(reference_size=7300)
Epoch: 0 -- 920.0000000000001 generations
     start     end      growth_rate |     0        1        2    
   -------- --------       -------- | -------- -------- -------- 
0 |1.45e+04 1.45e+04              0 |     0     2.5e-05  7.8e-06 
1 | 3.4e+04 1.03e+03         0.0038 |  2.5e-05     0    3.11e-05 
2 |4.59e+04    554           0.0048 |  7.8e-06 3.11e-05     0    

Events @ generation 920.0000000000001
   - Mass migration: lineages move from 2 to 1 with probability 1.0
   - Migration rate change to 0 everywhere
   - Migration rate change for (0, 1) to 0.00015
   - Migration rate change for (1, 0) to 0.00015
   - Population parameter change for 1: initial_size -> 1861 growth_rate -> 0 
Epoch: 920.0000000000001 -- 2040.0000000000002 generations
     start     end      growth_rate |     0        1        2    
   -------- --------       -------- | -------- -------- -------- 
0 |1.45e+04 1.45e+04              0 |     0     0.00

([<msprime.simulations.PopulationConfiguration at 0x7fe3b72c0550>,
 [[0, 2.5e-05, 7.8e-06], [2.5e-05, 0, 3.11e-05], [7.8e-06, 3.11e-05, 0]],
 [{'type': 'mass_migration', 'time': 920.0, 'source': 2, 'dest': 1, 'proportion': 1.0},
  {'type': 'migration_rate_change', 'time': 920.0, 'rate': 0, 'matrix_index': None},
  {'type': 'migration_rate_change', 'time': 920.0, 'rate': 0.00015, 'matrix_index': (0, 1)},
  {'type': 'migration_rate_change', 'time': 920.0, 'rate': 0.00015, 'matrix_index': (1, 0)},
  {'type': 'population_parameters_change', 'time': 920.0, 'growth_rate': 0, 'initial_size': 1861, 'population': 1},
  {'type': 'mass_migration', 'time': 2040.0, 'source': 1, 'dest': 0, 'proportion': 1.0},
  {'type': 'population_parameters_change', 'time': 5920.0, 'growth_rate': None, 'initial_size': 7300, 'population': 0}])